# March Madness Prediction

## Overview

### Goal
Submissions are based on the Brier Score, the goal will be to minimize the brier score between the predicted probabilities and the actual game outcomes. The Brier score measures the accuracy of probablistic predition, in this case the mean square error. 

The brier score can be thought of as a cost function that measures the average squared difference between the predicted probabilities and the actual outcomes.

$$
Brier = \frac{1}{N} \sum_{i=1}^{N} (p_i - o_i)^2
$$

where $p_i$ is the predicted probability of the event and $o_i$ is the actual outcome. The Brier score can span across all items in a set of N predictions.

Therefore, minimizing the Brier score will result in a more accurate prediction.



## Import Libraries
Numpy for numerical operations
Pandas for data manipulation
Matplotlib, Seaborn, Plotly for plotting



In [2]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import xgboost as xgb
import sklearn as sk

## Load Data

Set up a data dictionary that will store the data for each file, this will make it easier to access data from the csvs


In [11]:
data_dir = 'data/'
data = {}

files = glob.glob(data_dir + '*.csv')
for file in files:
    filename = file.split('/')[-1].split('\\')[-1].split('.')[0]
    try:
        data[filename] = pd.read_csv(file, encoding='latin-1')
    except Exception as e:
        print(f"Warning: Could not load {filename}: {e}")



Now that we have the data loaded, lets start by loading team and team spelling data.


In [14]:
teams = pd.concat([data['MTeams'], data['WTeams']])
teams_spelling = pd.concat([data['MTeamSpellings'], data['WTeamSpellings']])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
print(teams.head())

   TeamID     TeamName  FirstD1Season  LastD1Season  TeamNameCount
0    1101  Abilene Chr         2014.0        2025.0              3
1    1102    Air Force         1985.0        2025.0              2
2    1103        Akron         1985.0        2025.0              1
3    1104      Alabama         1985.0        2025.0              1
4    1105  Alabama A&M         2000.0        2025.0              2


Add season and turney reluts, both compact and detailed. There is a chance we only end up using the detailed results

In [17]:
season_compact_results = pd.concat([data['MRegularSeasonCompactResults'], data['WRegularSeasonCompactResults']]).assign(ST='S')
season_detailed_results = pd.concat([data['MRegularSeasonDetailedResults'], data['WRegularSeasonDetailedResults']]).assign(ST='S')
tourney_compact_results = pd.concat([data['MNCAATourneyCompactResults'], data['WNCAATourneyCompactResults']]).assign(ST='T')
tourney_detailed_results = pd.concat([data['MNCAATourneyDetailedResults'], data['WNCAATourneyDetailedResults']]).assign(ST='T')


In [23]:
# Create a summary of the datasets
summary = pd.DataFrame({
    'Dataset': [
        'Regular Season Compact',
        'Regular Season Detailed',
        'Tournament Compact',
        'Tournament Detailed'
    ],
    'Shape': [
        season_compact_results.shape,
        season_detailed_results.shape,
        tourney_compact_results.shape,
        tourney_detailed_results.shape
    ],
    'Columns': [
        len(season_compact_results.columns),
        len(season_detailed_results.columns),
        len(tourney_compact_results.columns),
        len(tourney_detailed_results.columns)
    ],
    'Sample Years': [
        f"{season_compact_results['Season'].min()}-{season_compact_results['Season'].max()}",
        f"{season_detailed_results['Season'].min()}-{season_detailed_results['Season'].max()}",
        f"{tourney_compact_results['Season'].min()}-{tourney_compact_results['Season'].max()}",
        f"{tourney_detailed_results['Season'].min()}-{tourney_detailed_results['Season'].max()}"
    ]
})

# Display the summary with nice formatting
display(summary.style.set_properties(**{
    'border-color': 'black',
    'border-style': 'solid',
    'border-width': '1px',
    'text-align': 'center'
}))

# If you want to see the first few rows of each dataset, you can also do:
print("\nSample of Regular Season Compact Results:")
display(season_compact_results.head(3))

print("\nSample of Regular Season Detailed Results:")
display(season_detailed_results.head(3))

print("\nSample of Tournament Compact Results:")
display(tourney_compact_results.head(3))

print("\nSample of Tournament Detailed Results:")
display(tourney_detailed_results.head(3))

,Dataset,Shape,Columns,Sample Years
0,Regular Season Compact,"(325732, 9)",9,1985-2025
1,Regular Season Detailed,"(196362, 35)",35,2003-2025
2,Tournament Compact,"(4168, 9)",9,1985-2024
3,Tournament Detailed,"(2276, 35)",35,2003-2024



Sample of Regular Season Compact Results:


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST
0,1985,20,1228,81,1328,64,N,0,S
1,1985,25,1106,77,1354,70,H,0,S
2,1985,25,1112,63,1223,56,H,0,S



Sample of Regular Season Detailed Results:


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ST
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,S
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,S
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,S



Sample of Tournament Compact Results:


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST
0,1985,136,1116,63,1234,54,N,0,T
1,1985,136,1120,59,1345,58,N,0,T
2,1985,136,1207,68,1250,43,N,0,T



Sample of Tournament Detailed Results:


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ST
0,2003,134,1421,92,1411,84,N,1,32,69,...,14,31,17,28,16,15,5,0,22,T
1,2003,136,1112,80,1436,51,N,0,31,66,...,7,7,8,26,12,17,10,3,15,T
2,2003,136,1113,84,1272,71,N,0,31,59,...,14,21,20,22,11,12,2,5,18,T
